In [ ]:
!pip install -q wikipedia-api

  Preparing metadata (setup.py) ... done


In [ ]:
import wikipediaapi
import json

In [ ]:
USER_AGENT = "task7-semantic-search/1.0 (ozbekk.cagatay@gmail.com)"
# Türkçe ve İngilizce Wikipedia API objeleri

wiki_tr = wikipediaapi.Wikipedia(user_agent=USER_AGENT, language='tr')
wiki_en = wikipediaapi.Wikipedia(user_agent=USER_AGENT, language='en')

# Çekilecek makale başlık listeleri (örnek)
titles_tr = [
    "Yapay zeka"
]
titles_en = [
    "Artificial intelligence"]


In [ ]:
def get_page_text(wiki, title):
    page = wiki.page(title)
    return page.text if page.exists() else None

# Dokümanları topla
docs = []
for t in titles_tr:
    text = get_page_text(wiki_tr, t)
    if text:
        docs.append({"title": t, "lang": "tr", "text": text})

for t in titles_en:
    text = get_page_text(wiki_en, t)
    if text:
        docs.append({"title": t, "lang": "en", "text": text})

print(f"Toplam {len(docs)} doküman toplandı.")

Toplam 2 doküman toplandı.


In [ ]:
# Kaydet
output_path = "/content/raw_documents.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    for doc in docs:
        f.write(json.dumps(doc, ensure_ascii=False) + "\n")

print(f"Veriler kaydedildi: {output_path}")

Veriler kaydedildi: /content/raw_documents.jsonl


In [ ]:
# === Parametreler ===
RAW_PATH   = "/content/drive/MyDrive/raw_documents.jsonl"  # bir önceki adımın çıktısı
CHUNK_PATH = "/content/drive/MyDrive/chunks.jsonl"
MAX_TOKENS = 350    # ~200–500 arası önerilir
OVERLAP    = 50     # bağlam korumak için küçük overlap


In [ ]:
# === Chunklama Yardımcıları ===
import re, json, os, unicodedata

def slugify(title: str) -> str:
    s = unicodedata.normalize("NFKD", title).encode("ascii", "ignore").decode("ascii")
    s = re.sub(r"[^a-zA-Z0-9]+", "-", s).strip("-").lower()
    return s or "doc"

def sent_split(text: str):
    text = re.sub(r"\s+", " ", text.strip())
    # basit ve güçlü bir cümle bölücü: . ? ! ; : sonları + satır sonları
    sents = re.split(r"(?<=[\.\?\!\;\:])\s+|\n+", text)
    return [s for s in sents if s]

def tokenize(s: str):
    return s.split()

def chunkify_by_tokens(sents, max_tokens=350, overlap=50):
    chunks, cur, cur_len = [], [], 0
    for s in sents:
        toks = tokenize(s)
        L = len(toks)
        if cur_len + L <= max_tokens:
            cur.append(s); cur_len += L
        else:
            if cur:
                chunks.append(" ".join(cur))
                # overlap uygula
                if overlap > 0:
                    last_tokens = tokenize(" ".join(cur))[-overlap:]
                    cur = [" ".join(last_tokens)]
                    cur_len = len(last_tokens)
                else:
                    cur, cur_len = [], 0
            # bu cümle tek başına büyükse parçala
            if L > max_tokens:
                for i in range(0, L, max_tokens):
                    chunks.append(" ".join(toks[i:i+max_tokens]))
                cur, cur_len = [], 0
            else:
                cur.append(s); cur_len += L
    if cur:
        chunks.append(" ".join(cur))
    return chunks

In [ ]:
# === Oku, chunkla, yaz ===
docs = []
with open(RAW_PATH, "r", encoding="utf-8") as f:
    for line in f:
        docs.append(json.loads(line))

os.makedirs(os.path.dirname(CHUNK_PATH), exist_ok=True)
n_docs, n_chunks = 0, 0

with open(CHUNK_PATH, "w", encoding="utf-8") as w:
    for d in docs:
        title = d.get("title", f"doc{n_docs}")
        lang  = d.get("lang", "en")
        text  = d.get("text", "")
        if not text.strip():
            continue
        n_docs += 1
        doc_id = f"{slugify(title)}"
        sents  = sent_split(text)
        parts  = chunkify_by_tokens(sents, MAX_TOKENS, OVERLAP)
        for i, ch in enumerate(parts):
            rec = {
                "doc_id": doc_id,
                "chunk_id": i,
                "text": ch,
                "language": lang,
                "source": title
            }
            w.write(json.dumps(rec, ensure_ascii=False) + "\n")
            n_chunks += 1

print(f"Tamam. {n_docs} dokümandan toplam {n_chunks} chunk yazıldı -> {CHUNK_PATH}")

Tamam. 13 dokümandan toplam 395 chunk yazıldı -> /content/drive/MyDrive/chunks.jsonl


In [ ]:
import json, statistics

CHUNK_PATH = "/content/drive/MyDrive/chunks.jsonl"

samples, lengths, by_doc = [], [], {}
with open(CHUNK_PATH, "r", encoding="utf-8") as f:
    for k, line in enumerate(f):
        obj = json.loads(line)
        txt = obj["text"]
        tok = len(txt.split())
        lengths.append(tok)
        key = obj["doc_id"]
        by_doc[key] = by_doc.get(key, 0) + 1
        if len(samples) < 3:  # 3 örnek göster
            samples.append(obj)

print(f"Toplam chunk: {len(lengths)}")
if lengths:
    print(f"Ortalama token/chunk: {statistics.mean(lengths):.1f}")
    print(f"Medyan token/chunk:   {statistics.median(lengths):.1f}")
    print(f"Min/Max token:        {min(lengths)} / {max(lengths)}")

print("\nÖrnek 3 chunk:")
for s in samples:
    print(f"- {s['doc_id']}#{s['chunk_id']} [{s['language']}]: {s['text'][:200]}...")

print("\nDoküman başına chunk sayısı (ilk 10):")
for i, (k, v) in enumerate(sorted(by_doc.items(), key=lambda x: -x[1])[:10], start=1):
    print(f"{i:>2}. {k}: {v}")


Toplam chunk: 395
Ortalama token/chunk: 334.3
Medyan token/chunk:   340.0
Min/Max token:        140 / 350

Örnek 3 chunk:
- yapay-zeka#0 [tr]: Yapay zekâ (İngilizce: Artificial intelligence, AI); hesaplama sistemlerinin öğrenme, akıl, problem çözme, algılama ve karar verme gibi insan zekâsıyla tipik olarak ilişkilendirilen görevleri yerine g...
- yapay-zeka#1 [tr]: eserlerinde dışa vurmuştur. Karel Čapek, R.U.R. adlı tiyatro oyununda yapay zekâya sahip robotlar ile insanlığın ortak toplumsal sorunlarını ele alarak 1920 yılında yapay zekânın insan aklından bağıms...
- yapay-zeka#2 [tr]: birçok alanda kullanılmaktadır. Tarihçe Yapay zekâ tarihi, antik çağlarda, usta zanaatkarlar tarafından zeka veya bilinç kazandırılan yapay varlıklara ilişkin mitler, hikâyeler ve söylentilerle başlad...

Doküman başına chunk sayısı (ilk 10):
 1. london: 53
 2. ottoman-empire: 53
 3. economy-of-the-united-states: 51
 4. artificial-intelligence: 47
 5. osmanl-imparatorlugu: 46
 6. real-madrid-cf: 45
 7. b